### Author: Md Fahim Hasan
### Work Email: mdfahim.hasan@bayer.com

In [3]:
import h3
import os
import re
import s3fs
import json
import boto3
import base64
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import pandas_gbq as gbq
from shapely.geometry import Polygon
from google.cloud import bigquery
from sklearn.neighbors import BallTree

# Read me

__This scripts consist of all functions required for querying weather, soil, and elevation data from CSW. It also has the assiting functions required for the querying process.__

In [6]:
# generating credentials using service account information
from cswCredentials import svc_secret
from google.oauth2 import service_account

service_account_creds = json.loads(base64.b64decode(svc_secret))
bq_project = 'location360-datasets'
credentials = service_account.Credentials.from_service_account_info(service_account_creds)
credentials

In [ ]:
# l0 : country name
# l1 : state/provinve/region name
# l2: county/district/division name

def extract_geom_info_by_country_state(countries_code, states, bq_project='location360-datasets'):
    """
    Extract geometry information (dataframe) with country_code and state name.
    
    params:
    country_code : list of str of country, e.g., ['MX'].
    states : list of str (multiple states) with city names.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried result.
    """ 
   
    geom_query_df = pd.DataFrame()
    for country_code, state in zip(countries_code, states):

        query = f"""
                SELECT l0_name as country, l0_iso_code as country_code, l1_name as state_province, l2_name as county_district, l3_name as subdistrict, geog as geometry
                FROM location360-datasets.geopolitical.global_l3 as l3
                WHERE l3.l0_iso_code = '{country_code}'
                AND l3.l1_name IN ('{state}')
                """
        temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
        geom_query_df = pd.concat([geom_query_df, temp_query])
        
    return geom_query_df


def extract_L3_geom_by_country_state(output_shapefile, country_code, state_name,
                                     bq_project='location360-datasets'):
    """
    Extract L3 geometry information with country code, state name and save as shapefile.
    
    params:
    output_shapefile : Filepath of output shapefile.
    country code : Country code for example 'US'.
    state_name : State/district name for example 'California'.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried result. ALso, saves the geodataframe as shapefile.
    """   
    query = f"""
            SELECT l0_name as country, l0_iso_code as country_code, l1_name as state_province, l2_name as county_district, l3_name as city, geog as geometry
            FROM location360-datasets.geopolitical.global_l3 as l3
            WHERE l3.l0_iso_code = '{country_code}'
            AND l3.l1_name IN ('{state_name}')
            """
    geom_query_df = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    geom_query_df['geometry'] =  geom_query_df['geometry'].apply(wkt.loads) # converting to wkt is required to create a GeoDataframe
    geom_query_gdf = gpd.GeoDataFrame(geom_query_df, geometry='geometry')
    
    # saving 
    geom_query_gdf.to_file(output_shapefile)
        
    return geom_query_gdf


def extract_geom_info_by_country_state_city(info_df, county_ID_col='country_id', l1_name_col='state', lat_col='latitude', 
                                            lon_col='longitude', bq_project='location360-datasets'):
    """
    Extract geometry information (dataframe) with country_code, state name, and city's lat lon information.
    
    params:
    info_df : pandas dataframe/csv filepath with information like country_code, state_name, and lat-lon.
    county_ID_col : column name with country_code in the given dataframe.
    l1_name_col : column name with level 1 admin name (state/province) in the given dataframe.
    lat_col : column name with latitude in the given dataframe.
    lon_col : column name with longitude in the given dataframe.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried result.
    """ 
    if isinstance(info_df, str) and '.csv' in info_df:
        info_df = pd.read_csv(info_df)
    country_code = info_df[county_ID_col]
    l1_name = info_df[l1_name_col]
    lat = info_df[lat_col]
    lon = info_df[lon_col]
        
    geom_query_df = pd.DataFrame()
    for country_code, state, x, y in zip(country_code, l1_name, lon, lat):
        query = f"""
                SELECT l0_name as country, l0_iso_code as country_code, l1_name as state_province, l2_name as county_district, geom as geometry
                FROM location360-datasets.geopolitical.global_l2 as l2
                WHERE l2.l0_iso_code = '{country_code}'
                AND l2.l1_name IN ('{state}')
                AND ST_Intersects(geom, ST_GEOGPOINT({x},{y}))
                """
        temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
        geom_query_df = pd.concat([geom_query_df, temp_query])
        
    return geom_query_df

def get_min_max_bound_of_state(country_code, state_name, bq_project='location360-datasets'):
    """
    Extract minx, miny, maxx, maxy bounds with country_code and state_name.
    
    params:
    country_code : str of country code, e.g., ['MX'].
    state_name : str of state name.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: minx, miny, maxx, maxy bounds as a tuple.
    """ 
    
    query = f"""
        SELECT l0_name as country, l0_iso_code as country_code, l1_name as state_province, geom as geometry
        FROM location360-datasets.geopolitical.global_l1 as l1
        WHERE l1.l0_iso_code = '{country_code}'
        AND l1.l1_name IN ('{state_name}')
        """
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads) # converting to wkt is required to create a GeoDataframe
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')
    bounds_df = query_gdf['geometry'].bounds
    
    minx, miny, maxx, maxy = bounds_df['minx'].values[0], bounds_df['miny'].values[0], bounds_df['maxx'].values[0], \
    bounds_df['maxy'].values[0]
    
    return minx, miny, maxx, maxy


def get_min_max_bound_of_county(country_code, county_name, bq_project='location360-datasets'):
    """
    Extract minx, miny, maxx, maxy bounds with country_code and county_name.
    
    params:
    country_code : str of country code, e.g., ['MX'].
    county_name : str of county_name.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: minx, miny, maxx, maxy bounds as a tuple.
    """ 

    query = f"""
            SELECT l0_name as country, l0_iso_code as country_code, l1_name as state, l2_name as county, geom as geometry
            FROM location360-datasets.geopolitical.global_l2 as l2
            WHERE l2.l0_iso_code = '{country_code}'
            AND l2.l2_name IN ('{county_name}')
            """
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)

    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads) # converting to wkt is required to create a GeoDataframe
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')
    bounds_df = query_gdf['geometry'].bounds
    
    minx, miny, maxx, maxy = bounds_df['minx'].values[0], bounds_df['miny'].values[0], bounds_df['maxx'].values[0], \
    bounds_df['maxy'].values[0]
    
    return minx, miny, maxx, maxy

In [1]:
def create_shapefile_for_state(country_code, state_name, output_folder, savename, bq_project='location360-datasets'):
    """
    Create shapefile with country_code and state_name.
    
    params:
    country_code : str of country code, e.g., ['MX'].
    state_name : str of state_name.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried shapefile.
    """ 
        
    query = f"""
        SELECT l0_name as country, l0_iso_code as cntry_code, l1_name as state, geom as geometry
        FROM location360-datasets.geopolitical.global_l1 as l1
        WHERE l1.l0_iso_code = '{country_code}'
        AND l1.l1_name IN ('{state_name}')
        """
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads) # converting to wkt is required to create a GeoDataframe
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')
    
    makedirs([output_folder])
    savefile = os.path.join(output_folder, savename)
    query_gdf.to_file(savefile)
    
    return query_gdf

def create_shapefile_for_county(country_code, county_name, output_folder, savename, bq_project='location360-datasets'):
    """
    Create shapefile with country_code and county_name.
    
    params:
    country_code : str of country code, e.g., ['MX'].
    county_name : str of county_name.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried shapefile.
    """ 
    
    query = f"""
            SELECT l0_name as country, l0_iso_code as cntry_code, l1_name as state, l2_name as county, geom as geometry
            FROM location360-datasets.geopolitical.global_l2 as l2
            WHERE l2.l0_iso_code = '{country_code}'
            AND l2.l2_name IN ('{county_name}')
            """
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads)  # converting to wkt is required to create a GeoDataframe
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')

    makedirs([output_folder])
    savefile = os.path.join(output_folder, savename)
    query_gdf.to_file(savefile)

    return query_gdf

def clip_grids_by_admin(grids_file, admin_file, output_folder, savename):
    """
    Clip a shapefile/geodataframe with another shapefile//geodataframe.
    
    
    *****the grids_file and admin_file both has to be same type. Either shapefile path/geodataframe*****
    
    params:
    grids_file : shapefile path/geodataframe with twc_grid/era5_rid information.
    admin_file : shapefile path/geodataframe used to clip the grids_file.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    
    returns: geopandas dataframe of cliiped shapefile/geodataframe.
    """
    
    if '.shp' not in grids_file:  # the grids_file and admin_file both has to be same type. Either shapefile path/geodataframe
        grids_df = grids_file
        admin_df = admin_file
    else:
        grids_df = gpd.read_file(grids_file)
        admin_df = gpd.read_file(admin_file)

        
    clipped_gdf = gpd.clip(grids_df['geometry'], admin_df['geometry'])
    clipped_gdf = gpd.GeoDataFrame(clipped_gdf, geometry='geometry')
    clipped_gdf = clipped_gdf.join(grids_df, on=None, how='left', lsuffix='', rsuffix='R')  # merging lost grids_df info to the clipped grids 
    clipped_gdf = clipped_gdf.drop(columns=['geometry', 'geometryR'])
    clipped_gdf = gpd.GeoDataFrame(clipped_gdf, geometry=gpd.points_from_xy(clipped_gdf.lon, clipped_gdf.lat), 
                                   crs="EPSG:4326")
    clipped_gdf = clipped_gdf.dropna()
    clipped_gdf = clipped_gdf.reset_index()
    
    makedirs([output_folder])
 
    if '.shp' not in savename:
        savename = savename + '.shp'
        
    savefile = os.path.join(output_folder, savename)
    clipped_gdf.to_file(savefile)
    
    return clipped_gdf

In [1]:
def add_geometry_to_h3(row):
    """
    create polygon geometry for h3 index.
    
    params:
    row : dataframe row. 
    
    returns: polygon geometry for h3 index.
    """
    points = h3.h3_to_geo_boundary(row['h3'], True)
    
    return Polygon(points)

def make_h3_hid_using_geodataframe(geom_df, output_folder, savename, h3_level=10, polygon_col='geometry'):
    """
    Create h3 index and hid using geometry info from a geodataframe.
    
    params:
    geom_df : A geodataframe with polygon geometry.
    output_folder : Filepath of output folder to save h3 and hid information as a shapefile.
    savename : Name of h3_hid shapefile that will be saved.
    h3_level : h3 level. Default set to 10. 
    polygon_col : polygon column in the geodataframe. Default set to 'geometry'.
    
    returns: A geodataframe with h3 and hid information.
    """
    
    if not isinstance(geom_df, gpd.geodataframe.GeoDataFrame):
        geom_df['geometry'] = geom_df[polygon_col].apply(wkt.loads) 
        geom_df_gpd = gpd.GeoDataFrame(geom_df, geometry='geometry')

    else:
        geom_df_gpd = geom_df
        
    h3_hid_df = pd.DataFrame()
    for poly in geom_df_gpd['geometry']:
        query = f"""
                SELECT environmental_data_cube.h3_2_hid(h3, {h3_level}) as hids, h3 as h3
                FROM UNNEST(environmental_data_cube.generate_h3_by_wkt_polygon('{poly}', {h3_level})) as h3;
                """
        result_query = gbq.read_gbq(query, project_id='location360-datasets', credentials=credentials)
        h3_hid_df = pd.concat([h3_hid_df, result_query])

    h3_hid_df['geometry'] = h3_hid_df.apply(add_geometry_to_h3, axis=1)
    h3_hid_gdf = gpd.GeoDataFrame(h3_hid_df, geometry='geometry')
    
    makedirs([output_folder])
    savefile = os.path.join(output_folder, savename)
    h3_hid_gdf.to_file(savefile)
    
    return h3_hid_gdf

In [1]:
def read_h3_parquet_as_geodataframe(parquet_file, h3_geometry_file, save=False, output_folder=None, savename=None):
    """
    Read parquet file with h3 information and save it as a geodataframe.
    
    params:
    parquet_file : Filepath of parquet file. Must have h3 information.
    h3_geometry-file: Filepath of h3 geometry file. Must have matching h3 information with the parquet file and geometry info.
    save : Set to true if want to save data as geodataframe/shapefile.
    output_folder : str of output folder to save the data. Default set to None.
    savename : str of name of the shapefile. Default set to None.
    
    returns: geopandas dataframe with data information.
    
    """
    df_parq = pd.read_parquet(parquet_file) # must have h3 information
    df_h3 = gpd.read_file(h3_geometry_file) # must have matching h3 information with the parquet file and geometry info
    
    df_compiled = df_parq.merge(df_h3, on='h3', how='inner')
    gdf_compiled = gpd.GeoDataFrame(df_compiled, geometry='geometry')
    
    if save:
        makedirs([output_folder])
        savefile = os.path.join(output_folder, savename)
        gdf_compiled.to_file(savefile)
    
    return gdf_compiled

def read_parquet_as_geodataframe(parquet_file, grid_geometry_file, save=False, output_folder=None, savename=None):
    """
    Read parquet file with twc/era5 grid information and save it as a geodataframe.
    
    params:
    parquet_file : Filepath of parquet file. Must have twc grid information.
    grid_geometry-file: Filepath of twc/era5 grid geometry file. Must have matching twc grid information with the 
                            parquet file and geometry info.
    save : Set to true if want to save data as geodataframe/shapefile. Large files will have error (dieing kernel). 
           Better to not save when facing such issues.
    output_folder : str of output folder to save the data. Default set to None.
    savename : str of name of the shapefile. Default set to None.
    
    returns: geopandas dataframe with data information.
    
    """
    df_parq = pd.read_parquet(parquet_file) # must have twc/era5 grid information
    df_grid = gpd.read_file(grid_geometry_file) # must have matching twc grid information with the parquet file and geometry info
    
    df_compiled = df_parq.merge(df_grid, on='grid_id', how='inner')
    gdf_compiled = gpd.GeoDataFrame(df_compiled, geometry='geometry')
    
    if save:
        makedirs([output_folder])
        savefile = os.path.join(output_folder, savename)
        gdf_compiled.to_file(savefile)
    
    return gdf_compiled

## Weather Data Query Codes

In [1]:
def query_twc_grids_for_specific_bounds(minx, miny, maxx, maxy, dataset_id, output_folder, savename, 
                                        bq_project='location360-datasets'):
    """
    Query and extract TWC grid ids as a geodataframe using minx, miny, maxx, maxy bounds.
    
    params:
    minx, miny, maxx, maxy: minx, miny, maxx, maxy bounds of a region.
    dataset_id : str of ID of dataset in Google Cloud. 
                 'location360-datasets.historical_weather.twc_cod_grids' for all dataset except precipitation.
                 'location360-datasets.historical_weather.twc_hires_precip_grids' for precipitation data.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried results with twc_grids and their geometry information.
    """
    
    bbox_wkt = f'POLYGON(({minx} {miny}, {minx} {maxy}, {maxx} { maxy}, {maxx} {miny}, {minx} {miny}))' # assigning min-max bound information to wkt format 
    
    query = f"""
    SELECT grid_id, lat, lon, geom as geometry, elevation, time_zone
    FROM {dataset_id} as twc_grids
    WHERE ST_INTERSECTS(geom, ST_GEOGFROMTEXT('{bbox_wkt}'))
    """
    
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads) # converting to wkt is required to create a GeoDataframe
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')
    
    makedirs([output_folder])
    savefile = os.path.join(output_folder, savename)
    query_gdf.to_file(savefile)
    
    return query_gdf   


def query_twc_weather_by_grid(grid_shp, date, dataset_id, output_folder, savename,
                              date_range=None, query_breaks=25,
                              var_columns = ['max_temperature', 'min_temperature', 'avg_wind_speed', 
                                             'avg_relative_humidity', 'avg_atmospheric_pressure'],
                              save_as_gpd=False, bq_project='location360-datasets'):
    """
    Query and extract TWC weather data for a specific date or date_range.
    
    params:
    grid_shp : shapefile path of twc grids.
    date : str of date for which to download data. If date is assinged set date_range to None. 
    dataset_id : str of ID of dataset in Google Cloud. 
                 'location360-datasets.historical_weather.twc_historical_metric_daily' for all dataset except precipitation.
                 'location360-datasets.historical_weather.twc_high_resolution_precipitation_metric_daily' for precipitation data.
    date_range : list of str of dates for which to download data. If date_range is assinged set date to None.
    query_breaks : Number of breaks to make in twc_grids for faster querying.
    var_columns : list of variables to download from twc dataset.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    save_as_gpd : Set to True if want to save queried data as a geodataframe. Default set to False.
    
    returns: pandas/geopandas dataframe of queried results.
    """
    
    var_rename_dict = {'total_precipitation': 'total_precip', 'max_temperature': 'max_temp', 'min_temperature': 'min_temp', 
                       'avg_wind_speed': 'avg_wind_speed', 'avg_relative_humidity': 'avg_Rhumid', 
                       'avg_atmospheric_pressure': 'avg_atmPress', 'avg_total_cloud_cover': 'avg_Cloud',
                       'total_downward_solar_radiation': 'TotDown_SR', 'max_downward_solar_radiation': 'MaxDown_SR'}
    
    twc_df = gpd.read_file(grid_shp)
    
    # rearranging df for quicker query. Each query will ask data for multiple grids.
    length = len(twc_df)
    query_breaks = np.linspace(start=0, stop=length, num=query_breaks, dtype='int')

    df_grids = list(twc_df['grid_id'])
    grids_stacked = []

    for i in range(len(query_breaks)-1):
        grids_new = df_grids[query_breaks[i] : query_breaks[i+1]]
        grids_stacked.append(grids_new)

    dict_grids = {'grid_id': grids_stacked}
    grid_df = pd.DataFrame(dict_grids)

    # Querying Data
    var_columns = ','.join(var_columns)

    twc_weather_query = pd.DataFrame()
    if date_range is None:
        for idx, row in grid_df.iterrows():
            grid_tup = tuple(row['grid_id'])
            query = f"""
                    SELECT grid_id, date, {var_columns}
                    FROM {dataset_id} as twc_historic_daily
                    WHERE grid_id IN {grid_tup} 
                    AND Date(date) = '{date}'
                    """
            temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
            twc_weather_query = pd.concat([twc_weather_query, temp_query])
            
    else: # set date to None
        date1, date2 = date_range[0], date_range[1] 
        for idx, row in grid_df.iterrows():
            grid_tup = tuple(row['grid_id'])
            query = f"""
                    SELECT grid_id, date, {var_columns}
                    FROM {dataset_id} as twc_historic_daily
                    WHERE grid_id IN {grid_tup} 
                    AND Date(date) BETWEEN '{date1}' AND '{date2}'  
                    """
            temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
            twc_weather_query = pd.concat([twc_weather_query, temp_query])
            
    makedirs([output_folder])
    if save_as_gpd:
        # Converting query to Geopandas
        twc_weather_query = twc_weather_query.merge(twc_df, on='grid_id', how='left')
        twc_weather_query = twc_weather_query.drop(columns=['index'])
        twc_weather_df = gpd.GeoDataFrame(twc_weather_query, geometry='geometry')
        twc_weather_df = twc_weather_df.rename(columns=var_rename_dict)
        twc_weather_df['date'] = twc_weather_df['date'].astype('str')  # google cloud date data type can't be read into dateformat in pandas. Converting to str.
        
        savefile = os.path.join(output_folder, savename)
        twc_weather_df.to_file(savefile)
        
    else:
        # Formatting dataframe
        twc_weather_df = twc_weather_query.rename(columns=var_rename_dict)
        twc_weather_df['date'] = twc_weather_df['date'].astype('str')  # google cloud date data type can't be read into dateformat in pandas. Converting to str.
        
        savefile = os.path.join(output_folder, savename)
        twc_weather_df.to_parquet(savefile)


    return twc_weather_df

def query_era5_grids_for_specific_bounds(minx, miny, maxx, maxy, output_folder, savename, bq_project='location360-datasets'):
    """
    Query and extract ERA5 grid ids as a geodataframe using minx, miny, maxx, maxy bounds.
    
    params:
    minx, miny, maxx, maxy: minx, miny, maxx, maxy bounds of a region.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried results with twc_grids and their geometry information.
    """
    
    bbox_wkt = f'POLYGON(({minx} {miny}, {minx} {maxy}, {maxx} { maxy}, {maxx} {miny}, {minx} {miny}))'
    
    query = f"""
    SELECT grid_id, lat, lon, time_zone, geom as geometry
    FROM location360-datasets.historical_weather.era5_grids as era5_grids
    WHERE ST_INTERSECTS(geom, ST_GEOGFROMTEXT('{bbox_wkt}'))
    """
    
    query_result = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
    query_result['geometry'] =  query_result['geometry'].apply(wkt.loads)
    query_gdf = gpd.GeoDataFrame(query_result, geometry='geometry')
    
    savefile = os.path.join(output_folder, savename)
    query_gdf.to_file(savefile)
    
    return query_gdf 

def query_era5_weather_by_grid(grid_shp, date, output_folder, savename,
                               date_range=None, query_breaks=25,
                               var_columns = ['total_precipitation', 'max_temperature', 'min_temperature', 
                                              'max_dew_point_temperature', 'min_dew_point_temperature', 'avg_dew_point_temperature',
                                              'max_wind_speed', 'min_wind_speed', 'avg_wind_speed', 'avg_wind_direction', 
                                              'max_relative_humidity', 'min_relative_humidity', 'avg_relative_humidity', 
                                              'max_atmospheric_pressure', 'min_atmospheric_pressure', 'avg_atmospheric_pressure', 
                                              'avg_snow_depth','avg_total_cloud_cover', 'total_downward_solar_radiation', 
                                              'max_downward_solar_radiation', 'total_net_solar_radiation', 'max_net_solar_radiation',
                                              'avg_soil_temperature_level_1', 'avg_soil_moisture_level_1',                                               'eto', 'etr'],
                              save_as_gpd=False, bq_project='location360-datasets', ):
    """
    Query and extract ERA5 weather data for a specific date or date_range.
    
    params:
    grid_shp : shapefile path of era5 grids.
    date : str of date for which to download data. If date is assinged set date_range to None. 
    date_range : list of str of dates for which to download data. If date_range is assinged set date to None.
    query_breaks : Number of breaks to make in era5_grids for faster querying.
    var_columns : list of variables to download from era5 dataset.
    output_folder : str of output folder to save the data. 
    savename : str of name of the shapefile.
    save_as_gpd : Set to True if want to save queried data as a geodataframe. Default set to False.
    bq_project : Bigquery project name. Default set to 'location360-datasets'.
    
    returns: geopandas dataframe of queried results.
    """
    
    var_rename_dict = {'total_precipitation': 'total_precip', 'max_temperature': 'max_temp', 'min_temperature': 'min_temp', 
                       'max_dew_point_temperature': 'max_dew_temp', 'min_dew_point_temperature': 'min_dew_temp', 
                       'avg_dew_point_temperature': 'avg_dew_temp', 'avg_wind_direction': 'avg_wind_dir',
                       'max_relative_humidity': 'max_Rhumid', 'min_relative_humidity': 'min_Rhumid', 
                       'avg_relative_humidity': 'avg_Rhumid', 'max_atmospheric_pressure': 'max_atmPress', 
                       'min_atmospheric_pressure': 'min_atmPress',
                       'avg_atmospheric_pressure': 'avg_atmPress', 'avg_total_cloud_cover': 'avg_Cloud',
                       'total_downward_solar_radiation': 'TotDown_SR', 'max_downward_solar_radiation': 'MaxDown_SR', 
                       'total_net_solar_radiation': 'TotNet_SR', 'max_net_solar_radiation': 'MaxNet_SR',
                       'avg_soil_temperature_level_1': 'avg_ST_L1', 'avg_soil_temperature_level_2': 'avg_ST_L2',
                       'avg_soil_temperature_level_3': 'avg_ST_L3', 'avg_soil_temperature_level_4': 'avg_ST_L4',
                       'avg_soil_moisture_level_1': 'avg_SM_L1', 'avg_soil_moisture_level_2':'avg_SM_L2',
                       'avg_soil_moisture_level_3': 'avg_SM_L3', 'avg_soil_moisture_level_4': 'avg_SM_L4'}
    
    era5_df = gpd.read_file(grid_shp)
    
    # rearranging df for quicker query. Each query will ask data for multiple grids.
    length = len(era5_df)
    query_breaks = np.linspace(start=0, stop=length, num=query_breaks, dtype='int')

    df_grids = list(era5_df['grid_id'])
    grids_stacked = []

    for i in range(len(query_breaks)-1):
        grids_new = df_grids[query_breaks[i] : query_breaks[i+1]]
        grids_stacked.append(grids_new)

    dict_grids = {'grid_id': grids_stacked}
    grid_df = pd.DataFrame(dict_grids)

    # Querying Data
    var_columns = ','.join(var_columns)

    era5_weather_query = pd.DataFrame()
    if date_range is None:
        for idx, row in grid_df.iterrows():
            grid_tup = tuple(row['grid_id'])
            query = f"""
                    SELECT grid_id, date, {var_columns}
                    FROM `location360-datasets.historical_weather.era5_metric_daily` as era5_metric_daily
                    WHERE grid_id IN {grid_tup} 
                    AND Date(date) = '{date}'
                    """
            temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
            era5_weather_query = pd.concat([era5_weather_query, temp_query])
            
    else: # set date to None
        date1, date2 = date_range[0], date_range[1] 
        for idx, row in grid_df.iterrows():
            grid_tup = tuple(row['grid_id'])
            query = f"""
                    SELECT grid_id, date, {var_columns}
                    FROM `location360-datasets.historical_weather.era5_metric_daily` as era5_metric_daily
                    WHERE grid_id IN {grid_tup} 
                    AND Date(date) BETWEEN '{date1}' AND '{date2}'  
                    """
            temp_query = gbq.read_gbq(query, project_id=bq_project, credentials=credentials)
            era5_weather_query = pd.concat([era5_weather_query, temp_query])
            
    if save_as_gpd:
        # Converting query to Geopandas
        era5_weather_query = era5_weather_query.merge(era5_df, on='grid_id', how='left')
        era5_weather_query = era5_weather_query.drop(columns=['index'])
        era5_weather_df = gpd.GeoDataFrame(era5_weather_query, geometry='geometry')
        era5_weather_df = era5_weather_df.rename(columns=var_rename_dict)
        era5_weather_df['date'] = era5_weather_df['date'].astype('str')
    
        savefile = os.path.join(output_folder, savename)
        era5_weather_df.to_file(savefile)
        
    else:
        # Formatting dataframe
        era5_weather_df = era5_weather_query.rename(columns=var_rename_dict)
        era5_weather_df['date'] = era5_weather_df['date'].astype('str')  # google cloud date data type can't be read into dateformat in pandas. Converting to str.

        savefile = os.path.join(output_folder, savename)
        era5_weather_df.to_parquet(savefile)
    
    return era5_weather_df

## Soil Data Query Codes

In [ ]:
def query_soil_data_on_hid_h3(hid_h3_df, output_folder, savename, query_breaks=500, hid_col='hids', h3_col='h3',
            soil250_cols = ['awcts_depth_0cm', 'awcts_depth_5cm', 'awcts_depth_15cm', 'awcts_depth_30cm', 
                            'awcts_depth_60cm', 'awcts_depth_100cm','wwp_depth_0cm', 'wwp_depth_5cm', 'wwp_depth_15cm', 
                            'wwp_depth_30cm', 'wwp_depth_60cm', 'wwp_depth_100cm'],
            soil250_V2_cols = ['nitrogen_0_5cm_mean', 'nitrogen_5_15cm_mean', 'nitrogen_15_30cm_mean', 'nitrogen_30_60cm_mean', 
                               'nitrogen_60_100cm_mean', 'nitrogen_100_200cm_mean', 'soc_0_5cm_mean', 'soc_5_15cm_mean', 
                               'soc_15_30cm_mean', 'soc_30_60cm_mean', 'soc_60_100cm_mean', 'soc_100_200cm_mean', 
                               'bulk_density', 'om', 'pH', 'sand', 'silt', 'clay', 'cec', 'soilTexture'],
            download_from = 'soil250'):
    
    
    """
    Queries soil250 V1 and V2 data by hids and h3.
    
    :param hid_h3_df : A geodataframe with h3 and hid info in columns.
    :param output_folder :  Filepath of output folder to save the queried dataframe and parquet.
    :param savename : Name of the parquet file. Should include '.parquet'.
    :param query_breaks : Number of breaks to make in twc_grids for faster querying.
    :param hid_col : Name of column in the hid_h3_df with 'hid' info. Default set to 'hids'.
    :param h3_col : Name of column in the hid_h3_df with 'h3' info. Default set to 'h3'.
    :param soil250_cols : List of column (data) names to download from soil250 V1 dataset. 
                          For downloading data from soil250 V1 dataset, set download_from = 'soil250'.
    :param soil250_V2_cols : List of column (data) names to download from soil250 V2 dataset. 
                             For downloading data from soil250 V1 dataset, set download_from = 'soil250_V2'.
    :param download_from : Set to 'soil250' to download data from soil250 V1 dataset.
                           Set to 'soil250_V2' to download data from soil250 V2 dataset.
    
    returns : geopandas dataframe of queried result.
    """
    
    rename_dict = {'awcts_depth_0cm': 'awct_0cm', 'awcts_depth_5cm': 'awct_5cm', 'awcts_depth_15cm': 'awct_15cm', 
                   'awcts_depth_30cm': 'awct_30cm', 'awcts_depth_60cm': 'awct_60cm', 'awcts_depth_100cm': 'awct_100cm',
                   'wwp_depth_0cm': 'wwp_0cm', 'wwp_depth_5cm': 'wwp_5cm', 'wwp_depth_15cm': 'wwp_15cm',
                   'wwp_depth_30cm': 'wwp_30cm', 'wwp_depth_60cm': 'wwp_60cm', 'wwp_depth_100cm': 'wwp_100cm',
                   'nitrogen_0_5cm_mean': 'nit0_5', 'nitrogen_5_15cm_mean': 'nit5_15', 'nitrogen_15_30cm_mean': 'nit15_30', 
                   'nitrogen_30_60cm_mean': 'nit30_60', 'nitrogen_60_100cm_mean': 'nit60_100', 
                   'nitrogen_100_200cm_mean': 'nit100_200', 
                   'soc_0_5cm_mean': 'soc0_5', 'soc_5_15cm_mean': 'soc5_15', 'soc_15_30cm_mean': 'soc15_30', 
                   'soc_30_60cm_mean': 'soc30_60', 'soc_60_100cm_mean': 'soc60_100', 
                   'soc_100_200cm_mean':'soc100_200', 'bulk_density': 'BulkDense', 'soilTexture': 'soiltext'}
    
    # Deciding the indices for query chunks. Each chunk will be queried at a time
    length = len(hid_h3_df['hids'])
    query_breaks = np.linspace(start=0, stop=length, num=query_breaks, dtype='int')

    # Making a list of hids/h3 from the input hid_h3_df
    hids = list(hid_h3_df['hids'])
    h3 = list(hid_h3_df['h3'])

    # splitting hids and h3 to multiple chunks. Each chunk of hid/h3 will be queried at a time
    hids_stacked = []
    h3_stacked = []
    for i in range(len(query_breaks)-1):
        hids_new = hids[query_breaks[i] : query_breaks[i+1]]
        h3_new = h3[query_breaks[i] : query_breaks[i+1]]
        hids_stacked.append(hids_new)
        h3_stacked.append(h3_new)
    
    # making a dataframe where each row consists of number of h3/hids to be queried together
    dict_hid_h3 = {'hids': hids_stacked, 'h3': h3_stacked}
    df = pd.DataFrame(dict_hid_h3)
    
    # setting up query column and location based on soil250 original and V2 sources
    if download_from == 'soil250':
        soil_cols = ','.join(soil250_cols)
        location = 'location360-datasets.environmental_data_cube.isric_global_soil_250'
    
    elif download_from == 'soil250_V2':
        soil_cols = ','.join(soil250_V2_cols)
        location = 'location360-datasets.environmental_data_cube.isric_global_soil_250_v2'
    
    # Looping through the stacked df and querying by chunks
    final_soil_query_df = pd.DataFrame()
    for idx, row in df.iterrows():
        hid_tup = tuple(row['hids'])
        h3_tup = tuple(row['h3'])
        
        # query from soil250
        query = f"""
                SELECT h3_index_10 as h3, hid, {soil_cols}
                FROM {location}
                WHERE hid in {hid_tup} and h3_index_10 in {h3_tup}
                """
        soil250_query = gbq.read_gbq(query, project_id='location360-datasets', credentials=credentials)
        final_soil_query_df = pd.concat([final_soil_query_df, soil250_query])
    
    final_soil_query_df = final_soil_query_df.rename(columns=rename_dict)
    savefile = os.path.join(output_folder, savename)
    final_soil_query_df.to_parquet(savefile)

    return final_soil_query_df

# Elevation Data Query Codes

In [ ]:
def query_srtm_elevation_data_on_hid_h3(hid_h3_df, output_folder, savename, query_breaks=500, hid_col='hids', h3_col='h3'):
    """
    Queries SRTM DEM data by hids and h3.
    
    :param hid_h3_df : A geodataframe with h3 and hid info in columns.
    :param output_folder :  Filepath of output folder to save the queried dataframe and parquet.
    :param savename : Name of the parquet file. Should include '.parquet'.
    :param query_breaks : Number of breaks to make in twc_grids for faster querying.
    :param hid_col : Name of column in the hid_h3_df with 'hid' info. Default set to 'hids'.
    :param h3_col : Name of column in the hid_h3_df with 'h3' info. Default set to 'h3'.
    
    returns : geopandas dataframe of queried result.
    """
    # Deciding the indices for query chunks. Each chunk will be queried at a time
    length = len(hid_h3_df['hids'])
    query_breaks = np.linspace(start=0, stop=length, num=query_breaks, dtype='int')

    # Making a list of hids/h3 from the input hid_h3_df
    hids = list(hid_h3_df['hids'])
    h3 = list(hid_h3_df['h3'])

    # splitting hids and h3 to multiple chunks. Each chunk of hid/h3 will be queried at a time
    hids_stacked = []
    h3_stacked = []
    for i in range(len(query_breaks)-1):
        hids_new = hids[query_breaks[i] : query_breaks[i+1]]
        h3_new = h3[query_breaks[i] : query_breaks[i+1]]
        hids_stacked.append(hids_new)
        h3_stacked.append(h3_new)

    # making a dataframe where each row consists of number of h3/hids to be queried together
    dict_hid_h3 = {'hids': hids_stacked, 'h3': h3_stacked}
    df = pd.DataFrame(dict_hid_h3)
    
    # Looping through the stacked df and querying by chunks
    final_elev_query_df = pd.DataFrame()
    for idx, row in df.iterrows():
        hid_tup = tuple(row['hids'])
        h3_tup = tuple(row['h3'])
        
        # query from soil250
        query = f"""
                SELECT h3, hids, elevation
                FROM `location360-datasets.environmental_data_cube.global_srtm_30m_topography`
                WHERE hids in {hid_tup} and h3 in {h3_tup}
                """
        temp_elev_query = gbq.read_gbq(query, project_id='location360-datasets', credentials=credentials)
        final_elev_query_df = pd.concat([final_elev_query_df, temp_elev_query])
    
    savefile = os.path.join(output_folder, savename)
    final_elev_query_df.to_parquet(savefile)

    return final_elev_query_df

## Others

In [2]:
def makedirs(directory_list):
    """
    Make directory (if not exists) from a list of directory.

    :param directory_list: A list of directories to create.

    :return: None.
    """
    for directory in directory_list:
        if not os.path.exists(directory):
            os.makedirs(directory)
            
def make_folder_in_s3_bucket(new_folder_path, bucket_name='data-pipeline-env-model'):
    """
    Make directory/folder in AWS S3 Bucket.
    
    params:
    new_folder_path : Folder path to create in the S3 bucket. Have to be like this "Main_folder/subfolder"
    bucket_name : S3 bucket name. Default set to 'data-pipeline-env-model'
    
    returns: None.
    """
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=(new_folder_path+'/'))